<a href="https://colab.research.google.com/github/Marco-Minniti/Bayesian_Network_scratch/blob/main/BN_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np

<h4><b>Node Class</b></h4>

Represents a node in a Bayesian network. It has two parameters in its constructor: `parents` and `probs`.

The `parents` parameter is a list that contains the names of the parent nodes of the current node that influences the probability distribution of the current node.

The `probs` parameter is a dictionary that represents the conditional probability distribution of the current node given its parent nodes. The keys of the dictionary are tuples that represent the different combinations of parent node values, and the values are dictionaries that represent the probabilities of different outcomes for the current node.

In [10]:
class Node:
    def __init__(self, parents, probs):
        self.parents = parents
        self.probs = probs

<h4><b>Network Structure</b></h4>
The Bayesian network is rapresented by a dictionary.

Each `key` in the dictionary is the **name of a node** and the corresponding `value` is an instance of the class **Node** defined by two main attributes described in the class above so:

**• parents** that is a list of parent nodes  
**• probs** that is a dictionary that maps combinations of parent values to probability distributions of the node's possible values.

In [11]:
# Shuffled net
net = {

    'Dinner': Node(['Lunch', 'Sport'], {('healthy', 'yes'): {'healthy': 0.9, 'unhealthy': 0.1}, ('healthy', 'no'): {'healthy': 0.6, 'unhealthy': 0.4}, ('unhealthy', 'yes'): {'healthy': 0.7, 'unhealthy': 0.3}, ('unhealthy', 'no'): {'healthy': 0.4, 'unhealthy': 0.6}}),
    'Alarm': Node([], {'yes': 0.9, 'no': 0.1}),
    'Breakfast': Node(['Wakeup', 'Alarm'], {('early', 'yes'): {'yes': 0.9, 'no': 0.1}, ('early', 'no'): {'yes': 0.9, 'no': 0.1}, ('late', 'yes'): {'yes': 0.6, 'no': 0.4}, ('late', 'no'): {'yes': 0.6, 'no': 0.4}}),
    'Toilet': Node(['Breakfast'], {('yes'): {'quick': 0.7, 'slow': 0.3}, ('no'): {'quick': 0.4, 'slow': 0.6}}),
    'Work': Node(['Shower'], {('quick'): {'productive': 0.9, 'unproductive': 0.1}, ('slow'): {'productive': 0.6, 'unproductive': 0.4}}),
    'Meeting': Node(['Lunch'], {('healthy'): {'yes': 0.8, 'no': 0.2}, ('unhealthy'): {'yes': 0.4, 'no': 0.6}}),
    'Sport': Node(['Work', 'Meeting'], {('productive', 'yes'): {'yes': 0.9, 'no': 0.1}, ('productive', 'no'): {'yes': 0.6, 'no': 0.4}, ('unproductive', 'yes'): {'yes': 0.7, 'no': 0.3}, ('unproductive', 'no'): {'yes': 0.4, 'no': 0.6}}),
    'Wakeup': Node([], {'early': 0.5, 'late': 0.5}),
    'Lunch': Node(['Work'], {('productive'): {'healthy': 0.7, 'unhealthy': 0.3}, ('unproductive'): {'healthy': 0.4, 'unhealthy': 0.6}}),
    'Shower': Node(['Breakfast', 'Toilet'], {('yes', 'quick'): {'quick': 0.9, 'slow': 0.1}, ('yes', 'slow'): {'quick': 0.7, 'slow': 0.3}, ('no', 'quick'): {'quick': 0.6, 'slow': 0.4}, ('no', 'slow'): {'quick': 0.4, 'slow': 0.6}}),

}

Network graphical rapresentation:
![Img](https://drive.google.com/uc?export=view&id=1wWBoCsZds9D1VcPyEamcL3covJLuq5oV)


<h4><b>Network Class</b></h4>

The constructor of the class takes as input a `net` dictionary representing the Bayesian network. Each key in the dictionary is the name of a node and the corresponding value is an instance of the Node class.

**sample_node(self, node, evidence)**: This method samples from a specific node given the evidence. The evidence is a dictionary that maps node names to their sampled values. The method returns the sampled value from the specified node.

**topological_sort(self)**: Method that performs a topological sorting of nodes in the network. Topological sorting is a linear list of nodes such that for every direct arc from node A to node B, A appears before B in the list. This is useful for making inferences about the network.

**ancestral_sampling(self)**: This method performs ancestral sampling from the network, generating samples from a Bayesian network. It begins by sampling from the root nodes of the network and proceeds in topological order, sampling from each node conditioned by its parents.

In [12]:
class Network:
    def __init__(self, net):
        self.net = net

    def topological_sort(self):
        visited = set()
        topo_order = []
        def visit(node_name):
            if node_name in visited:
                return
            visited.add(node_name)
            for parent in self.net[node_name].parents:
                visit(parent)
            topo_order.append(node_name)
        for node_name in self.net:
            visit(node_name)
        return topo_order

    def sample_node(self, node, evidence):
        probs = node.probs # probability of the selected node

        parents_values=[]
        if len(node.parents) > 1:
            for parent in node.parents: # scroll through the parents of the selected node (the first one is skipped because it has no parents)
                parents_values.append(evidence[parent])
            probs = node.probs[tuple(parents_values)] # take the probabilities of the selected node
        else:
            if len(node.parents) == 1:
                for parent in node.parents:
                    probs = node.probs[evidence[parent]]
        return np.random.choice(list(probs.keys()), p=list(probs.values())) # randomly choose among the keys (of the dictionary probs).....based on the probabilities specified by the values. (remember that the list chosen from where to extract the key depends on the value taken by the parent/parents)

    def ancestral_sampling(self):
        evidence = {}
        sampled_nodes = {}
        topo_order = self.topological_sort()  # add this line to get the topological order
        for node_name in topo_order:  # iterate over nodes in topological order
            node = self.net[node_name]
            sampled_value = self.sample_node(node, evidence)
            sampled_nodes[node_name] = sampled_value
            evidence[node_name] = sampled_value
        return sampled_nodes

<h4><b>Script</b></h4>

Generating an instance of the **Network** class using net and obtaining a **topological ordering** of the nodes in the network to be able to make inference about the network.

In [13]:
network = Network(net)
topo_order = network.topological_sort()

<h4><b>Test</b></h4>

Running 10 times the ancestral sampling. Each of them represents a possible sequence of events given the structure of the network and the probability distributions of the nodes. This can be useful in understanding how the variables in the network interact with each other.

In [15]:
for _ in range(10):
    print(network.ancestral_sampling())

{'Wakeup': 'late', 'Alarm': 'no', 'Breakfast': 'yes', 'Toilet': 'quick', 'Shower': 'quick', 'Work': 'productive', 'Lunch': 'healthy', 'Meeting': 'yes', 'Sport': 'yes', 'Dinner': 'healthy'}
{'Wakeup': 'late', 'Alarm': 'yes', 'Breakfast': 'no', 'Toilet': 'quick', 'Shower': 'quick', 'Work': 'productive', 'Lunch': 'unhealthy', 'Meeting': 'no', 'Sport': 'no', 'Dinner': 'unhealthy'}
{'Wakeup': 'early', 'Alarm': 'yes', 'Breakfast': 'yes', 'Toilet': 'slow', 'Shower': 'quick', 'Work': 'productive', 'Lunch': 'healthy', 'Meeting': 'yes', 'Sport': 'yes', 'Dinner': 'unhealthy'}
{'Wakeup': 'early', 'Alarm': 'yes', 'Breakfast': 'yes', 'Toilet': 'slow', 'Shower': 'slow', 'Work': 'unproductive', 'Lunch': 'unhealthy', 'Meeting': 'no', 'Sport': 'no', 'Dinner': 'unhealthy'}
{'Wakeup': 'late', 'Alarm': 'yes', 'Breakfast': 'no', 'Toilet': 'quick', 'Shower': 'quick', 'Work': 'productive', 'Lunch': 'healthy', 'Meeting': 'yes', 'Sport': 'yes', 'Dinner': 'healthy'}
{'Wakeup': 'late', 'Alarm': 'no', 'Breakfast':